# One-way Repeated Measures ANOVA Example

### With tests of assumptions, ANOVA model, and post-hoc tests.

In this notebook I conduct a one-way repeated measures ANOVA analysis, including tests of assumptions (Mauchly's test of sphericity), and appropriate post-hoc tests.

In [1]:
# Importing the key software libraries that will be used in the analysis. 

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pingouin as pg

In [2]:
# Importing the dataset to be analysed
rm_df = pd.read_csv('HR Polygraph.csv')

rm_df

,id,Lie,Truth,Coloured_sqr,Framing
0,1,58.536483,84.858119,53.783338,1
1,2,74.968975,72.235336,75.913702,1
2,3,61.114431,91.722885,87.325517,1
3,4,60.214866,78.241833,77.854317,1
4,5,61.410426,72.986057,102.853567,1
...,...,...,...,...,...
75,76,45.824996,67.734263,51.065190,2
76,77,101.107714,70.883897,86.426843,2
77,78,85.011061,69.120016,111.991884,2
78,79,90.235426,57.992397,54.251209,2


### Tidying and cleaning the data

The data contains a between-subjects (independent groups) factor called Framing, and a repeated measures factor that is currently split over three columns (Lie, Truth, Coloured_sqr). These are measurements of mean arousal taken from participants at three different time points, when they lied, told the truth or completed a control task. Arousal was measured using the participants heart rate. The data are currently in short and wide format and it would be preferable to combine these three repeated measures columns into one variable for arousal and to create another categorical variable containing the labels (lie, truth, coloured squares) for each of these levels/ conditions of the independent variable. 

In [3]:
# First, I am going to create a groups object with the three variables of interest:

groups = rm_df[['Lie', 'Truth', 'Coloured_sqr']]

groups

,Lie,Truth,Coloured_sqr
0,58.536483,84.858119,53.783338
1,74.968975,72.235336,75.913702
2,61.114431,91.722885,87.325517
3,60.214866,78.241833,77.854317
4,61.410426,72.986057,102.853567
...,...,...,...
75,45.824996,67.734263,51.065190
76,101.107714,70.883897,86.426843
77,85.011061,69.120016,111.991884
78,90.235426,57.992397,54.251209


In [4]:
# Currently the data is short and wide with the three categories saved as three separate variables. 
# I need to convert this to a long and thin dataframe. 
# This can be done using the pandas melt() method
cat_cond = list(groups.columns)
groups_long = pd.melt(groups, value_vars = cat_cond, value_name = 'mean_arousal', ignore_index = False)

In [5]:
# Calling the new groups_long data object created above

groups_long

,variable,mean_arousal
0,Lie,58.536483
1,Lie,74.968975
2,Lie,61.114431
3,Lie,60.214866
4,Lie,61.410426
...,...,...
75,Coloured_sqr,51.065190
76,Coloured_sqr,86.426843
77,Coloured_sqr,111.991884
78,Coloured_sqr,54.251209


In [6]:
# Adding a identity variable for each participant to account for the structure in the data due to the participants contributing
# multiple data points. 

groups_long['participant'] = rm_df['id']

In [7]:
# Checking this variable has been appended. 

groups_long

,variable,mean_arousal,participant
0,Lie,58.536483,1
1,Lie,74.968975,2
2,Lie,61.114431,3
3,Lie,60.214866,4
4,Lie,61.410426,5
...,...,...,...
75,Coloured_sqr,51.065190,76
76,Coloured_sqr,86.426843,77
77,Coloured_sqr,111.991884,78
78,Coloured_sqr,54.251209,79


### Tests of Assumptions

With the data in an appropriate format, I will now use Mauchly's test of sphericity. This tests the assumption that variances of the differences of all possible pairs of within-subjects conditions are equal. If sphericity is violated, then variance calculations may be distorted resulting in an inflated F ratio. If the assumption of sphericity is not violated then we can assume that the data is appropriate for use with a parametric statistical test like ANOVA.

In [8]:
# Running Mauchly's test using the pg.sphericity method.

pg.sphericity(data = groups_long, dv = 'mean_arousal', subject = 'participant', within = 'variable')

SpherResults(spher=False, W=0.8817264823737025, chi2=9.818123777047587, dof=2, pval=0.0073794077923853025)

We can see from the above pingouin output for Mauchly's test that we have a significant result and the assumption of sphericity has been violated. Mauchly's: W = 0.88, X2(2) = 9.82, p = 0.007. 

As a consequence when running an ANOVA we will need to use some sort of robust p-value correction for the ANOVA test. For this analysis we will use the Greenhouse Geisser corrected p-value that is provided as part of the output for the pingouin RMAnova method. 

In [9]:
# Conducting the repeated measures ANOVA using the pingouin rm_anova method. 
# The parameters for this method are the data, the DV, the within subjects factor, the subject/ participant ID 
# (to account for multiple datapoints from the same participants), detailed = True returns the result as a pandas dataframe.

res = pg.rm_anova(data = groups_long, dv = 'mean_arousal', within = 'variable', subject = 'participant', detailed = True)

res

,Source,SS,DF,MS,F,p-unc,p-GG-corr,ng2,eps,sphericity,W-spher,p-spher
0,variable,1856.868502,2,928.434251,3.142515,0.045885,0.051769,0.025424,0.894236,False,0.881726,0.007379
1,Error,46680.000758,158,295.443043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The above ANOVA result contains a Greenhouse Geisser corrected p-value of 0.052. This falls just above the traditional alpha level for statistical significance of 0.05 used in classical statistics but can be reported as significant to two decimal places. As a consequence, here, I will assume we have a statistically significant result and perform post-hoc tests using the pingouin pairwise_tests method. I will also specify the Bonferroni correction as the method for controlling for Type 1 error when making multiple comparisons.   

In [10]:
# As we have a significant ANOVA result we would now want to perform post-hoc comparisons on all pairs of conditions 
# of the independent variable (questions). We can do this using pingouin's pairwise_tests method. Below I have used Bonferroni
# corrected pariwise comparisons.

post_res = pg.pairwise_tests(data = groups_long, dv = 'mean_arousal', within = 'variable', subject = 'participant', 
                            parametric = True, padjust = 'bonf', effsize = 'hedges')

post_res

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,variable,Coloured_sqr,Lie,True,True,-1.626385,79.0,two-sided,0.107850,0.323551,bonf,0.434,-0.257469
1,variable,Coloured_sqr,Truth,True,True,0.801622,79.0,two-sided,0.425176,1.000000,bonf,0.168,0.108861
2,variable,Lie,Truth,True,True,2.252364,79.0,two-sided,0.027075,0.081224,bonf,1.332,0.382085


Although we would not normally conduct post-hoc tests when the ANOVA result was not significant, the fact that the ANOVA model was close to the threshold alpha level of p = 0.05 meant that it was probably reasonable in this case to conduct post-hoc tests and identify differences between groups that may be statistically significant.

In this case examination of the corrected p-values (p-corr) for the pairwise comparisons suggests that there were no significant differences between the groups.

- Coloured Sq v Lie: t(79) = 1.63, p = 0.32 
- Coloured Sq v Truth: t(79) = 0.80, p = 1.00 
- Lie v Truth: t(79) = 2.25, p = 0.08